# Operaciones básicas en Spark
- Spark opera con colecciones **inmutables y distribuidas** de elementos, manipulándolos en paralelo
    - API estructurada: DataFrames y DataSets
    - API de bajo nivel: RDDs (ya obsoleto ya que el API estructurada es más eficiente y más de alto nivel)

-   Operaciones sobre estas colecciones
    -   Creación
    -   Transformaciones (ordenación, filtrado, etc.)
    -   Realización acciones para obtener resultados

-   Spark automáticamente distribuye los datos y paraleliza las operaciones

### Ejemplo: creación de un DataFrame a partir de un fichero CSV
En este ejemplo, Spark infiere el esquema de los datos de forma automática

  - Es preferible especificar el esquema de forma explícita, como veremos más adelante

También se especifica que la primera línea es la cabecera

In [5]:
%%sh
wget -q "https://raw.githubusercontent.com/dsevilla/tcdm-public/24-25/datos/2015-summary.csv"
ls -lh 2015-summary.csv
head 2015-summary.csv

-rw-r--r--  1 luisi  staff   6.9K Nov  5 16:42 2015-summary.csv
DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count
United States,Romania,15
United States,Croatia,1
United States,Ireland,344
Egypt,United States,15
United States,India,62
United States,Singapore,1
United States,Grenada,62
Costa Rica,United States,588
Senegal,United States,40


In [ ]:
%pip install pyspark

In [2]:
from pyspark.sql import SparkSession
# Creamos un objeto SparkSession (o lo obtenemos si ya está creado)
spark = SparkSession \
  .builder \
  .appName("Mi aplicacion") \
  .config("spark.alguna.opcion.de.configuracion", "algun-valor") \
  .master("local[*]") \
  .getOrCreate()

sc = spark.sparkContext

24/11/05 16:40:29 WARN Utils: Your hostname, MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 172.18.26.240 instead (on interface en0)
24/11/05 16:40:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/05 16:40:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
from pyspark.sql.dataframe import DataFrame

datosVuelos2015: DataFrame = (spark
    .read
    .option("inferSchema", "true")
    .option("header", "true")
    .csv("2015-summary.csv"))

In [7]:
datosVuelos2015.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: integer (nullable = true)



In [ ]:
datosVuelos2015.show()
print(datosVuelos2015.count()) # cada fila del dataframe es un objeto de tipo row, es decir, una tabla es una lista de filas

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|   15|
|       United States|            Croatia|    1|
|       United States|            Ireland|  344|
|               Egypt|      United States|   15|
|       United States|              India|   62|
|       United States|          Singapore|    1|
|       United States|            Grenada|   62|
|          Costa Rica|      United States|  588|
|             Senegal|      United States|   40|
|             Moldova|      United States|    1|
|       United States|       Sint Maarten|  325|
|       United States|   Marshall Islands|   39|
|              Guyana|      United States|   64|
|               Malta|      United States|    1|
|            Anguilla|      United States|   41|
|             Bolivia|      United States|   30|
|       United States|           Paraguay|    6|
|             Algeri

In [9]:
datosVuelos2015.show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+
only showing top 5 rows



### Rows

Las filas de un DataFrame son objetos de tipo `Row`

- API de Row en Python: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.Row.html

In [10]:
# Obtenemos las dos primeras fila del DataFrame
from pyspark.sql.types import Row

rows1_2: list[Row] = datosVuelos2015.take(2) # devuelve las 2 primeras filas del DataFrame como una lista de objetos Row
print(rows1_2)
print(type(rows1_2))
print(type(rows1_2[0]))

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15), Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1)]
<class 'list'>
<class 'pyspark.sql.types.Row'>


In [11]:
# Obtén la primera fila como un diccionario Python
print(rows1_2[0].asDict())
print(type(rows1_2[0].asDict()))

{'DEST_COUNTRY_NAME': 'United States', 'ORIGIN_COUNTRY_NAME': 'Romania', 'count': 15}
<class 'dict'>


### Particiones

Spark divide las filas DataFrame en un conjunto de particiones

-   El número de particiones por defecto es función del tamaño del cluster (número total de cores en todos los ejecutores) y del tamaño de los datos (número de bloques de los ficheros en HDFS)
-   Para RDDs se puede especificar otro valor en el momento de crearlos
-   También se puede modificar una vez creados

In [12]:
print("Número de particiones: {0}"
    .format(datosVuelos2015.rdd.getNumPartitions()))

# Creo un nuevo DataFrame con 4 particiones
datosVuelos2015_4P: DataFrame = datosVuelos2015.repartition(4)
print("Número de particiones: {0}"
    .format(datosVuelos2015_4P.rdd.getNumPartitions()))

Número de particiones: 1
Número de particiones: 4


### Transformaciones

Operaciones que transforman los datos

  - No modifican los datos de origen (*inmutabilidad*)
  - Se computan de forma “perezosa” (*lazyness*)

Dos tipos:

  - Transformaciones *estrechas* (narrow)
    - Cada partición de entrada contribuye a una única partición de salida
    - No se modifica el número de particiones
    - Normalmente se realizan en memoria
  - Transformaciones *anchas* (wide)
    - Cada partición de salida depende de varias (o todas) particiones de entrada
    - Suponen un barajado de datos
    - Pueden implicar un cambio en el número de particiones
    - Pueden suponer escrituras en disco

In [13]:
# Ejemplo de una transformación narrow
datosVuelos2015_EEUU: DataFrame = datosVuelos2015\
    .replace("United States", "Estados Unidos")

In [ ]:
# Ejemplo de una transformación wide
datosVuelos2015_Ord: DataFrame = datosVuelos2015_EEUU\
    .sort("count", ascending=False)
datosVuelos2015_Ord.cache() # cache() es una acción que guarda el DataFrame en memoria

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: int]

### Acciones

Obtienen un resultado, forzando a que se realicen las transformaciones pendientes

  - En el momento de disparar la *acción* se crea un *plan* con las transformaciones necesarias para obtener los datos solicitados
    - Se crea un Grafo Dirigido Acíclico (DAG) conectando las transformaciones
    - Spark optimiza ese grafo, para eliminar transformaciones innecesarias o unir las que sea posible
  - Las acciones traducen el DAG en un plan de ejecución

Tipos de acciones

  - Acciones para mostrar datos por consola
  - Acciones para convertir datos Spark en datos del lenguaje
  - Acciones para escribir datos a disco


In [15]:
# Ejemplo de acciones
print("Número de filas en la tabla: {0}"
    .format(datosVuelos2015_Ord.count()))

print(datosVuelos2015_Ord.take(3))

datosVuelos2015_Ord.show()

Número de filas en la tabla: 256
[Row(DEST_COUNTRY_NAME='Estados Unidos', ORIGIN_COUNTRY_NAME='Estados Unidos', count=370002), Row(DEST_COUNTRY_NAME='Estados Unidos', ORIGIN_COUNTRY_NAME='Canada', count=8483), Row(DEST_COUNTRY_NAME='Canada', ORIGIN_COUNTRY_NAME='Estados Unidos', count=8399)]
+------------------+-------------------+------+
| DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|
+------------------+-------------------+------+
|    Estados Unidos|     Estados Unidos|370002|
|    Estados Unidos|             Canada|  8483|
|            Canada|     Estados Unidos|  8399|
|    Estados Unidos|             Mexico|  7187|
|            Mexico|     Estados Unidos|  7140|
|    United Kingdom|     Estados Unidos|  2025|
|    Estados Unidos|     United Kingdom|  1970|
|             Japan|     Estados Unidos|  1548|
|    Estados Unidos|              Japan|  1496|
|           Germany|     Estados Unidos|  1468|
|    Estados Unidos| Dominican Republic|  1420|
|Dominican Republic|     Estados Un